# Battle of the Neighborhoods

## Introduction

This project is for who is planning to open a Coffee House in Ankara, Turkey. This project suggests the best locations for Coffee Houses in Ankara. Ankara is the capital of Turkey with a population of 5M. Turkish culture has deep roots in coffee. Also, it is getting popular because of its affordable prices. Coffee consumption has increased 13% and average consumption is 1.1 kg/person in 2018. 
This report explores which neighborhoods of Ankara have the most as well as the best Coffee Houses. Also, this project answers the questions “Where should I open an Coffee House?” and “Where should I stay If I want a tasty coffee?”

## Data

<ul>
<li>District of Ankara are obtained from https://en.wikipedia.org/wiki/Ankara_Province</li>

<li>Latitude and Longitude values are obtained by using "geocoder".</li>

<li>All data related to locations will be obtaine by using FourSquare API and Python Libraries.</li>
</ul>

In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
#!conda install -c conda-forge geopy --yes
import geocoder

In [2]:
wiki_link = 'https://en.wikipedia.org/wiki/Ankara_Province'
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:64.0) Gecko/20100101 Firefox/64.0'}
wiki_page = requests.get(wiki_link, headers = headers)
wiki_page

<Response [200]>

In [3]:
soup = BeautifulSoup(wiki_page.content, 'html.parser')
table = soup.find('table', {'class':'wikitable sortable'}).tbody

In [4]:
rows = table.find_all('tr')

In [5]:
columns = [i.text.replace('\n', '') for i in rows[0].find_all('th')]
columns

['District', 'Population (2017)', 'Area (km²)', 'Density (per km²)']

In [6]:
df_ankara = pd.DataFrame(columns = columns)

In [7]:
for i in range(1, len(rows)):
    tds = rows[i].find_all('td')
    
    if len(tds) == 7:
        values = [tds[0].text, tds[1].text, tds[2].text.replace('\n', ''.replace('\xa0','')), tds[3].text, tds[4].text.replace('\n', ''.replace('\xa0','')), tds[5].text.replace('\n', ''.replace('\xa0','')), tds[6].text.replace('\n', ''.replace('\xa0',''))]
    else:
        values = [td.text.replace('\n', '').replace('\xa0','') for td in tds]
        
        df_ankara = df_ankara.append(pd.Series(values, index = columns), ignore_index = True)

        df_ankara

In [8]:
df_ankara.head()

,District,Population (2017),Area (km²),Density (per km²)
0,Akyurt,32.863,369,89
1,Altındağ,371.366,123,3.019
2,Ayaş,12.289,1.041,12
3,Bala,21.682,1.851,12
4,Beypazarı,48.476,1.697,29


In [9]:
def get_latlng(arcgis_geocoder):
    
    lat_lng_coords = None
    
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Ankara, Turkey'.format(arcgis_geocoder))
        lat_lng_coords = g.latlng
    return lat_lng_coords

In [10]:
districts = df_ankara['District']    
coordinates = [get_latlng(districts) for districts in districts.tolist()]

In [11]:
df_ankara_loc = df_ankara

df_ankara_coordinates = pd.DataFrame(coordinates, columns = ['Latitude', 'Longitude'])
df_ankara_loc['Latitude'] = df_ankara_coordinates['Latitude']
df_ankara_loc['Longitude'] = df_ankara_coordinates['Longitude']

In [12]:
df_ankara_loc.drop(columns="Population (2017)", axis=1, inplace=True)

In [13]:
df_ankara_loc.drop(columns="Density (per km²)", axis=1, inplace=True)

In [14]:
df_ankara_loc.drop(columns="Area (km²)", axis=1, inplace=True)

In [15]:
df_ankara_loc.head()

,District,Latitude,Longitude
0,Akyurt,40.13082,33.08719
1,Altındağ,39.94171,32.85445
2,Ayaş,40.01516,32.33270
3,Bala,39.55391,33.12352
4,Beypazarı,40.16811,31.92052


In [16]:
import numpy as np
import json 
from geopy.geocoders import Nominatim 

import requests 
from pandas.io.json import json_normalize

import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes 
import folium

print("Libraries imported")

Libraries imported


In [17]:
from geopy.geocoders import Nominatim 

address = "Çankaya, Ankara"

geolocator = Nominatim(user_agent = "Ankara_explorer")

location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

print("The geographical coordinates of Ankara are {}, {}.".format(latitude, longitude))

The geographical coordinates of Ankara are 39.9207774, 32.854067.


In [18]:
map_ankara = folium.Map(location=[latitude, longitude], zoom_start=10)

for lat, lng, label in zip(df_ankara_loc["Latitude"], df_ankara_loc["Longitude"], df_ankara_loc["District"]):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=25,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.3,
        parse_html=False).add_to(map_ankara)  
    
map_ankara

In [19]:
CLIENT_ID = "FBMIJHTR42BI0I5JUWFNREL3NTHT553IQUQ3AM1NGMQDDSHJ"
CLIENT_SECRET = "OVH1MFLGAQG4O4IKBKKHQLZJTDBL2QNFSV2R1NKA4YUBINQC" 
VERSION = "20180605"

In [20]:
df_ankara_loc.loc [0, "District"]
df_ankara_loc.loc [0, "District"]

'Akyurt'

In [21]:
neighborhood_latitude = df_ankara_loc.loc[0, "Latitude"]
neighborhood_longitude = df_ankara_loc.loc[0, "Longitude"] 

neighborhood_name = df_ankara_loc.loc[0, "District"] 

print("Latitude and longitude values of the neighborhood {} are {}, {}.".format(neighborhood_name, neighborhood_latitude, neighborhood_longitude))

Latitude and longitude values of the neighborhood Akyurt are 40.13082000000003, 33.08719000000008.


In [22]:
LIMIT = 100
radius = 500

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=FBMIJHTR42BI0I5JUWFNREL3NTHT553IQUQ3AM1NGMQDDSHJ&client_secret=OVH1MFLGAQG4O4IKBKKHQLZJTDBL2QNFSV2R1NKA4YUBINQC&v=20180605&ll=40.13082000000003,33.08719000000008&radius=500&limit=100'

In [23]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5e277285d03993001b9408ad'},
 'response': {'headerLocation': 'Akyurt',
  'headerFullLocation': 'Akyurt, Ankara',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 15,
  'suggestedBounds': {'ne': {'lat': 40.13532000450003,
    'lng': 33.093064640939495},
   'sw': {'lat': 40.126319995500026, 'lng': 33.08131535906066}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '5049c781e4b0a7a32d840eb1',
       'name': 'Meşhur Köfteci Bodur',
       'location': {'address': 'Cumhuriyet Meydani',
        'crossStreet': 'Cumhuriyet Meydani',
        'lat': 40.13071902888477,
        'lng': 33.08677622815274,
        'labeledLatLngs': [{'label': 'display',
          'lat': 40.13071902888477,
          'lng': 33.08677622815274}],
        '

In [24]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [25]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON


filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]


nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)


nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head(20)

,name,categories,lat,lng
0,Meşhur Köfteci Bodur,Turkish Restaurant,40.130719,33.086776
1,Akyurt kültür parkı,Park,40.133123,33.085071
2,Yurtalan Restaurant,Kebab Restaurant,40.132672,33.084306
3,Beyazıt Sofrası,Steakhouse,40.130542,33.081901
4,Onur pide ve kebap salonu,Kebab Restaurant,40.130807,33.086422
5,Yurtalan,Turkish Restaurant,40.132743,33.084385
6,onur park restorant,Restaurant,40.132863,33.084868
7,Benliler Supermarket,Convenience Store,40.130252,33.081423
8,Akyurt Sofrası,Turkish Restaurant,40.130854,33.086709
9,Yurtalan Restaurant,Doner Restaurant,40.132641,33.084492


In [26]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

15 venues were returned by Foursquare.


In [27]:
def getNearbyVenues(names, latitudes, longitudes, radius=1500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['District', 
                  'Latitude', 
                  'Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [28]:
ankara_venues = getNearbyVenues(names=df_ankara_loc['District'],
                                   latitudes=df_ankara_loc['Latitude'],
                                   longitudes=df_ankara_loc['Longitude']
                                  )

Akyurt
Altındağ
Ayaş
Bala
Beypazarı
Çamlıdere
Çankaya
Çubuk
Elmadağ
Etimesgut
Evren
Gölbaşı
Güdül
Haymana
Kahramankazan
Kalecik
Keçiören
Kızılcahamam
Mamak
Nallıhan
Polatlı
Pursaklar
Sincan
Şereflikoçhisar
Yenimahalle
Urban (9 districts)Altındağ, Çankaya, Etimesgut, Gölbaşı, Keçiören, Mamak, Pursaklar, Sincan, Yenimahalle
TOTAL


In [29]:
print(ankara_venues.shape)
ankara_venues.head(250)

(1689, 7)


,District,Latitude,Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Akyurt,40.13082,33.08719,Meşhur Köfteci Bodur,40.130719,33.086776,Turkish Restaurant
1,Akyurt,40.13082,33.08719,Akyurt kültür parkı,40.133123,33.085071,Park
2,Akyurt,40.13082,33.08719,Yurtalan Restaurant,40.132672,33.084306,Kebab Restaurant
3,Akyurt,40.13082,33.08719,Beyazıt Sofrası,40.130542,33.081901,Steakhouse
4,Akyurt,40.13082,33.08719,Onur pide ve kebap salonu,40.130807,33.086422,Kebab Restaurant
5,Akyurt,40.13082,33.08719,CrossFit Center 34,40.124721,33.076126,Gym
6,Akyurt,40.13082,33.08719,Yurtalan,40.132743,33.084385,Turkish Restaurant
7,Akyurt,40.13082,33.08719,onur park restorant,40.132863,33.084868,Restaurant
8,Akyurt,40.13082,33.08719,Benliler Supermarket,40.130252,33.081423,Convenience Store
9,Akyurt,40.13082,33.08719,Akyurt Sofrası,40.130854,33.086709,Turkish Restaurant


In [30]:
ankara_venues.groupby("District").count()

,Latitude,Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
District,,,,,,
Akyurt,29,29,29,29,29,29
Altındağ,100,100,100,100,100,100
Ayaş,42,42,42,42,42,42
Bala,11,11,11,11,11,11
Beypazarı,79,79,79,79,79,79
Elmadağ,52,52,52,52,52,52
Etimesgut,98,98,98,98,98,98
Evren,4,4,4,4,4,4
Gölbaşı,100,100,100,100,100,100


In [31]:
print('There are {} unique venue categories.'.format(len(ankara_venues['Venue Category'].unique())))

There are 232 unique venue categories.


In [32]:
ankara_onehot = pd.get_dummies(ankara_venues[['Venue Category']], prefix="", prefix_sep="")

ankara_onehot['Neighborhood'] = ankara_venues['District'] 

fixed_columns = [ankara_onehot.columns[-1]] + list(ankara_onehot.columns[:-1])
ankara_onehot = ankara_onehot[fixed_columns]

ankara_onehot

,Neighborhood,ATM,Accessories Store,Advertising Agency,Airport Lounge,Antique Shop,Aquarium,Arcade,Art Gallery,Art Museum,...,Used Bookstore,Video Game Store,Vineyard,Volleyball Court,Warehouse Store,Watch Shop,Water Park,Wedding Hall,Women's Store,Yoga Studio
0,Akyurt,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Akyurt,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Akyurt,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Akyurt,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Akyurt,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,Akyurt,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,Akyurt,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,Akyurt,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,Akyurt,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,Akyurt,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [33]:
ankara_onehot.shape

(1689, 233)

In [34]:
ankara_grouped = ankara_onehot.groupby('Neighborhood').mean().reset_index()
ankara_grouped

,Neighborhood,ATM,Accessories Store,Advertising Agency,Airport Lounge,Antique Shop,Aquarium,Arcade,Art Gallery,Art Museum,...,Used Bookstore,Video Game Store,Vineyard,Volleyball Court,Warehouse Store,Watch Shop,Water Park,Wedding Hall,Women's Store,Yoga Studio
0,Akyurt,0.000000,0.00,0.00,0.000000,0.00,0.000000,0.000000,0.00,0.000000,...,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,Altındağ,0.000000,0.00,0.00,0.000000,0.03,0.000000,0.000000,0.02,0.010000,...,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,Ayaş,0.000000,0.00,0.00,0.000000,0.00,0.000000,0.000000,0.00,0.000000,...,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,Bala,0.000000,0.00,0.00,0.000000,0.00,0.000000,0.000000,0.00,0.000000,...,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,Beypazarı,0.000000,0.00,0.00,0.000000,0.00,0.000000,0.012658,0.00,0.000000,...,0.00,0.000000,0.000000,0.012658,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,Elmadağ,0.019231,0.00,0.00,0.000000,0.00,0.000000,0.019231,0.00,0.000000,...,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.019231,0.000000,0.000000,0.000000
6,Etimesgut,0.000000,0.00,0.00,0.000000,0.00,0.000000,0.040816,0.00,0.000000,...,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.010204
7,Evren,0.000000,0.00,0.00,0.000000,0.00,0.000000,0.000000,0.00,0.000000,...,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,Gölbaşı,0.000000,0.00,0.00,0.000000,0.00,0.000000,0.040000,0.00,0.000000,...,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.010000,0.000000,0.000000,0.000000
9,Güdül,0.000000,0.00,0.00,0.000000,0.00,0.000000,0.000000,0.00,0.000000,...,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [35]:
ankara_grouped.shape

(27, 233)

In [36]:
num_top_venues = 10

for hood in ankara_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = ankara_grouped[ankara_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Akyurt----
                venue  freq
0                Park  0.10
1  Turkish Restaurant  0.10
2    Kebab Restaurant  0.07
3                Farm  0.07
4              Garden  0.03
5          Food Court  0.03
6             Stadium  0.03
7       Big Box Store  0.03
8  Basketball Stadium  0.03
9    Basketball Court  0.03


----Altındağ----
                venue  freq
0               Hotel  0.11
1                Café  0.08
2      History Museum  0.07
3       Jewelry Store  0.06
4  Turkish Restaurant  0.06
5       Historic Site  0.05
6             Theater  0.04
7    Kebab Restaurant  0.03
8        Antique Shop  0.03
9          Steakhouse  0.03


----Ayaş----
                     venue  freq
0       Turkish Restaurant  0.12
1               Restaurant  0.07
2                     Lake  0.07
3                     Café  0.05
4                     Park  0.05
5                     Pool  0.05
6               Food Court  0.05
7        Convenience Store  0.02
8           Farmers Market  0.02
9  Co

In [37]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [38]:
num_top_venues = 10
indicators = ['st', 'nd', 'rd']

columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))
        
ankara_venues_sorted = pd.DataFrame(columns=columns)
ankara_venues_sorted['Neighborhood'] = ankara_grouped['Neighborhood']

for ind in np.arange(ankara_grouped.shape[0]):
    ankara_venues_sorted.iloc[ind, 1:] = return_most_common_venues(ankara_grouped.iloc[ind, :], num_top_venues)

ankara_venues_sorted.head(27)

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Akyurt,Park,Turkish Restaurant,Farm,Kebab Restaurant,Big Box Store,Stadium,Basketball Court,Basketball Stadium,Food Court,Shopping Mall
1,Altındağ,Hotel,Café,History Museum,Jewelry Store,Turkish Restaurant,Historic Site,Theater,Steakhouse,Antique Shop,Kebab Restaurant
2,Ayaş,Turkish Restaurant,Lake,Restaurant,Pool,Café,Food Court,Park,Kebab Restaurant,BBQ Joint,Farm
3,Bala,Electronics Store,Convenience Store,Turkish Restaurant,Mountain,Steakhouse,Bakery,Big Box Store,Business Service,Café,Soccer Stadium
4,Beypazarı,Turkish Restaurant,History Museum,Scenic Lookout,Restaurant,Bed & Breakfast,Bakery,Motel,Hotel,Tea Room,Plaza
5,Elmadağ,Café,Park,Convenience Store,Campground,Big Box Store,Bar,Dessert Shop,Kebab Restaurant,Jewelry Store,Forest
6,Etimesgut,Turkish Restaurant,Café,Steakhouse,Gym,Arcade,Restaurant,Dessert Shop,Shopping Mall,Kebab Restaurant,Park
7,Evren,River,Steakhouse,Grocery Store,Park,Yoga Studio,Fast Food Restaurant,Farmers Market,Farm,Event Space,Entertainment Service
8,Gölbaşı,Café,Turkish Restaurant,Park,Restaurant,Coffee Shop,Arcade,Breakfast Spot,Middle Eastern Restaurant,Kebab Restaurant,Dessert Shop
9,Güdül,Department Store,Kebab Restaurant,Bagel Shop,River,Athletics & Sports,Moving Target,Castle,Souvenir Shop,Botanical Garden,Farmers Market


In [39]:
from sklearn.cluster import KMeans

In [42]:
ankara_merged = df_ankara
ankara_merged['Cluster Labels'] = kmeans.labels_

ankara_merged = ankara_merged.join(ankara_venues_sorted.set_index('Neighborhood'), on='District')
ankara_venues_sorted.head(10)

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Akyurt,Park,Turkish Restaurant,Farm,Kebab Restaurant,Big Box Store,Stadium,Basketball Court,Basketball Stadium,Food Court,Shopping Mall
1,Altındağ,Hotel,Café,History Museum,Jewelry Store,Turkish Restaurant,Historic Site,Theater,Steakhouse,Antique Shop,Kebab Restaurant
2,Ayaş,Turkish Restaurant,Lake,Restaurant,Pool,Café,Food Court,Park,Kebab Restaurant,BBQ Joint,Farm
3,Bala,Electronics Store,Convenience Store,Turkish Restaurant,Mountain,Steakhouse,Bakery,Big Box Store,Business Service,Café,Soccer Stadium
4,Beypazarı,Turkish Restaurant,History Museum,Scenic Lookout,Restaurant,Bed & Breakfast,Bakery,Motel,Hotel,Tea Room,Plaza
5,Elmadağ,Café,Park,Convenience Store,Campground,Big Box Store,Bar,Dessert Shop,Kebab Restaurant,Jewelry Store,Forest
6,Etimesgut,Turkish Restaurant,Café,Steakhouse,Gym,Arcade,Restaurant,Dessert Shop,Shopping Mall,Kebab Restaurant,Park
7,Evren,River,Steakhouse,Grocery Store,Park,Yoga Studio,Fast Food Restaurant,Farmers Market,Farm,Event Space,Entertainment Service
8,Gölbaşı,Café,Turkish Restaurant,Park,Restaurant,Coffee Shop,Arcade,Breakfast Spot,Middle Eastern Restaurant,Kebab Restaurant,Dessert Shop
9,Güdül,Department Store,Kebab Restaurant,Bagel Shop,River,Athletics & Sports,Moving Target,Castle,Souvenir Shop,Botanical Garden,Farmers Market


In [59]:
ks = 3

ankara_grouped_clustering = ankara_grouped.drop('Neighborhood', 1)

kmeans = KMeans(n_clusters=ks, random_state=0).fit(ankara_grouped_clustering)

kmeans.labels_[1:10]

array([1, 1, 1, 1, 1, 1, 2, 1, 1])

In [60]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(ks)
ys = [i+x+(i*x)**2 for i in range(ks)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(ankara_merged['Latitude'], ankara_merged['Longitude'], ankara_merged['District'], ankara_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [61]:
ankara_merged.loc[ankara_merged['Cluster Labels'] == 0, ankara_merged.columns[[0] + list(range(4, ankara_merged.shape[1]))]]

,District,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
23,Şereflikoçhisar,Café,Plaza,Turkish Restaurant,Restaurant,Seafood Restaurant,Motel,Electronics Store,Fast Food Restaurant,Bar,Basketball Court


In [62]:
ankara_merged.loc[ankara_merged['Cluster Labels'] == 1, ankara_merged.columns[[0] + list(range(4, ankara_merged.shape[1]))]]

,District,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Akyurt,Park,Turkish Restaurant,Farm,Kebab Restaurant,Big Box Store,Stadium,Basketball Court,Basketball Stadium,Food Court,Shopping Mall
1,Altındağ,Hotel,Café,History Museum,Jewelry Store,Turkish Restaurant,Historic Site,Theater,Steakhouse,Antique Shop,Kebab Restaurant
2,Ayaş,Turkish Restaurant,Lake,Restaurant,Pool,Café,Food Court,Park,Kebab Restaurant,BBQ Joint,Farm
3,Bala,Electronics Store,Convenience Store,Turkish Restaurant,Mountain,Steakhouse,Bakery,Big Box Store,Business Service,Café,Soccer Stadium
4,Beypazarı,Turkish Restaurant,History Museum,Scenic Lookout,Restaurant,Bed & Breakfast,Bakery,Motel,Hotel,Tea Room,Plaza
5,Çamlıdere,Forest,Art Museum,Plaza,French Restaurant,Turkish Restaurant,Market,BBQ Joint,Farmers Market,Farm,Dog Run
6,Çankaya,Café,Bar,Bookstore,Optical Shop,Pub,Turkish Restaurant,Sandwich Place,Gym,Dessert Shop,Kebab Restaurant
8,Elmadağ,Café,Park,Convenience Store,Campground,Big Box Store,Bar,Dessert Shop,Kebab Restaurant,Jewelry Store,Forest
9,Etimesgut,Turkish Restaurant,Café,Steakhouse,Gym,Arcade,Restaurant,Dessert Shop,Shopping Mall,Kebab Restaurant,Park
10,Evren,River,Steakhouse,Grocery Store,Park,Yoga Studio,Fast Food Restaurant,Farmers Market,Farm,Event Space,Entertainment Service


In [63]:
ankara_merged.loc[ankara_merged['Cluster Labels'] == 2, ankara_merged.columns[[0] + list(range(4, ankara_merged.shape[1]))]]

,District,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
7,Çubuk,Café,Turkish Restaurant,Bistro,Athletics & Sports,Convenience Store,Gym / Fitness Center,Mobile Phone Shop,Garden,Grocery Store,Fast Food Restaurant


In [58]:
ankara_merged.loc[ankara_merged['Cluster Labels'] == 3, ankara_merged.columns[[0] + list(range(5, ankara_merged.shape[1]))]]

,District,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
